<img width=200 src="https://camo.githubusercontent.com/903f3cc51db134b8c9faed2ba2b18ffedff67ff2aafe75259cbde477b27d9b4f/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f7468756d622f652f65642f50616e6461735f6c6f676f2e7376672f3132303070782d50616e6461735f6c6f676f2e7376672e706e673f7261773d74727565"></img>

# Day-18 Pandas 大型資料處理與效能調校

* 範例目標：
  1. 實做效能調校
  2. 大型資料集處理
* 範例重點：
  1. 在這裡介紹 3 種加速方法針對 Pandas ，在 Python 中還有很多方式可以提升效能
  2. 欄位的型態降級有助於減少記憶體佔用空間

## 匯入套件

In [ ]:
# 載入 NumPy, Pandas 套件
import numpy as np
import pandas as pd
import time

# 檢查正確載入與版本
print(np)
print(np.__version__)
print(pd)
print(pd.__version__)

<module 'numpy' from 'D:\\anaconda3\\lib\\site-packages\\numpy\\__init__.py'>
1.19.2
<module 'pandas' from 'D:\\anaconda3\\lib\\site-packages\\pandas\\__init__.py'>
1.1.3


## 效能調校

有三個方法可以大幅減少程式的執行時間：
* 讀取資料型態選最快速的
* 多使用內建函數
* 向量化的資料處理

### 讀取資料型態選最快速的

* 以下四種資料型態進行實測，當然不同環境與不同資料會有所差距，不過資料越多改善會越明顯
* 發現讀取速度以 pkl 檔為最快，是平常讀 csv 的 6 倍速


| 文件格式 | 運行時間(mean±std) | 速度倍數(以csv為基準) |
|---------|:------------------:|:-------------------:|
| xlsx | 1min 19s ± 2.82s | 無視 |
|  csv | 582ms ± 16.6ms | 1 |
|  pkl | 98.4ms ± 1.9ms | 5.90 |
|  hdf | 120ms ± 1.79ms | 4.84 |

In [ ]:
score_df = pd.DataFrame([[1,50,80,70,'boy',1], 
              [2,60,45,50,'boy',2],
              [3,98,43,55,'boy',1],
              [4,70,69,89,'boy',2],
              [5,56,79,60,'girl',1],
              [6,60,68,55,'girl',2],
              [7,45,70,77,'girl',1],
              [8,55,77,76,'girl',2],
              [9,25,57,60,'girl',1],
              [10,88,40,43,'girl',3],
              [11,25,60,45,'boy',3],
              [12,80,60,23,'boy',3],
              [13,20,90,66,'girl',3],
              [14,50,50,50,'girl',3],
              [15,89,67,77,'girl',3]],columns=['student_id','math_score','english_score','chinese_score','sex','class'])
score_df

student_id  math_score  english_score  chinese_score   sex  class
0            1          50             80             70   boy      1
1            2          60             45             50   boy      2
2            3          98             43             55   boy      1
3            4          70             69             89   boy      2
4            5          56             79             60  girl      1
5            6          60             68             55  girl      2
6            7          45             70             77  girl      1
7            8          55             77             76  girl      2
8            9          25             57             60  girl      1
9           10          88             40             43  girl      3
10          11          25             60             45   boy      3
11          12          80             60             23   boy      3
12          13          20             90             66  girl      3
13          14          50             50             50  girl      3
14          15          89             67             77  girl      3

### 多使用內建函數

* 使用 agg 和 transform 進行操作時，儘量使用 Python 的內建函式，能夠提高執行效率
* 可看出 groupby + agg + 內建函數是最快的，因為 pandas 的內建函數皆有經過加速後的算法
* 如果非得需要用到自己的函式，那盡量使用 agg 會比 transform 來的快速

#### agg vs. transform

* [最佳] agg 使用 Python 的內建函式

In [ ]:
star_time = time.time()
score_df.groupby('class').agg('mean')
end_time = time.time()
end_time - star_time

0.0032105445861816406

* agg 使用自定義函式

In [ ]:
star_time = time.time()
score_df.groupby('class').agg(lambda x: x.mean())
end_time = time.time()
end_time - star_time

0.01680469512939453

* transform 使用 Python 的內建函式

In [ ]:
star_time = time.time()
score_df.groupby('class').transform('mean')
end_time = time.time()
end_time - star_time

0.019934415817260742

* transform 使用自定義函式

In [ ]:
star_time = time.time()
score_df.groupby('class').transform(lambda x: x.mean())
end_time = time.time()
end_time - star_time

0.027081966400146484

### 向量化的資料處理

#### [isin](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html)

* 採用 isin() 篩選出對應資料是最快的，速度快是因為它採用向量化的資料處理方式(還有其他方式)

* 用 list 方式搜索

In [ ]:
#篩選出對應資料
score_df1 = score_df.copy()
star_time = time.time()
score_df1['Pass_math'] = [i>=60 for i in score_df1.math_score]
end_time = time.time()
end_time - star_time

0.0021719932556152344

* 用 DataFrame column 方式搜索

In [ ]:
score_df1 = score_df.copy()
star_time = time.time()
score_df1['Pass_math'] = score_df1.math_score>=60
end_time = time.time()
end_time - star_time

0.02496051788330078

* 用自定義式搜索

In [ ]:
score_df2 = score_df.copy()
star_time = time.time()
score_df2['Pass_math'] = score_df2.math_score.apply(lambda x : x>=60)
end_time = time.time()
end_time - star_time

0.0016407966613769531

* [最佳] 用isin()

In [ ]:
score_df3 = score_df.copy()
star_time = time.time()
score_df3['Pass_math'] = score_df3.math_score.isin(range(60, 100))
end_time = time.time()
end_time - star_time

0.0014753341674804688

## 大型資料集處理
欄位的型態降級有助於減少記憶體佔用空間

* 遇到大資料集時，常有記憶體不足的問題，還有速度上變慢，此時我們可以將欄位的型態降級，不需要存太多元素在一個數字中
* 首先先生成大資料，因為改善部分不同所以分成浮點數 float 與整數 int 的資料集，可以看到不管浮點數還是整數都佔了 800128bytes

In [ ]:
np.random.randint(3,9,10)

array([3, 7, 5, 5, 8, 4, 5, 8, 6, 3])

In [ ]:
float_data = pd.DataFrame(np.random.uniform(0,5,100000).reshape(1000,100))
int_data = pd.DataFrame(np.random.randint(0,1000,100000).reshape(1000,100))
int_data.memory_usage(deep=True).sum(), float_data.memory_usage(deep=True).sum()

(800128, 800128)

* 將整數型態 int 改成 uint 減少記憶體正用空間，使用前 800128bytes，使用後剩下 200128bytes
* 原有 100 個欄位是 int64，經過 downcast 變成了 100 個欄位的 uint16

In [ ]:
downcast_int = int_data.apply(pd.to_numeric, downcast='unsigned')
int_data.memory_usage(deep=True).sum(), downcast_int.memory_usage(deep=True).sum()

(800128, 200128)

In [ ]:
compare_int = pd.concat([int_data.dtypes, downcast_int.dtypes],axis=1)
compare_int.columns = ['before', 'after']
compare_int.apply(pd.value_counts)

before  after
uint16     NaN  100.0
int64    100.0    NaN

* 將浮點數型態 float64 改成 float32 減少記憶體正用空間，使用前 800128bytes，使用後剩下 400128bytes

In [ ]:
downcast_float = float_data.apply(pd.to_numeric, downcast='float')
float_data.memory_usage(deep=True).sum(), downcast_float.memory_usage(deep=True).sum()

(800128, 400128)

* 原有 100 個欄位是 float64，經過 downcast 變成了 100 個欄位的 float32

In [ ]:
compare_int = pd.concat([float_data.dtypes, downcast_float.dtypes],axis=1)
compare_int.columns = ['before', 'after']
compare_int.apply(pd.value_counts)

before  after
float32     NaN  100.0
float64   100.0    NaN

## 參考資料

* [效能調校](https://iter01.com/454098.html)
* [常見的 Pandas 性能優化方法](https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/716987/)
* [四大性能優化方法](https://www.boxuegu.com/news/2222.html)
* [Pandas 性能優化-基礎篇](http://flypython.com/ml/142.html)
* [Pandas 性能優化-進階篇](http://flypython.com/ml/139.html)
* [pandas.DataFrame.memory_usage](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.memory_usage.html)